In [1]:
from db1_preprocess_utils import *
from feature_extraction import *
from experiment_one_utils import *

2023-09-03 13:38:40.365582: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-03 13:38:40.641720: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-03 13:38:41.336356: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bmis/anaconda3/envs/tf/lib/python3.9/site-packages/nvidia/cudnn/lib:/home/bmis/anaconda3/envs/tf/lib/:
2023-09-03 13:38:41.336592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [2]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from scipy.signal import stft


"""
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

"""

'\ngpus = tf.config.list_physical_devices(\'GPU\')\nif gpus:\n  # Restrict TensorFlow to only use the first GPU\n  try:\n    tf.config.set_visible_devices(gpus[0], \'GPU\')\n    logical_gpus = tf.config.list_logical_devices(\'GPU\')\n    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")\n  except RuntimeError as e:\n    # Visible devices must be set before GPUs have been initialized\n    print(e)\n\n'

In [3]:
czzzzzzzz

signal_type='raw'
input_type='raw'
n_channels=64
low_cut=20
high_cut=500
order=6 
window_size=50
overlap=50
fs=2048

whiten=False
center=True
extend=False
extend_size=0
normalize=False
mu=0

start_subject = 1
total_subject = 20
session=[1,2]
ratio = 0.2

noise_db = [5,10,15]
std = 1
stochastic_depth_rate = 0.1

In [4]:
data, label, X_test, y_test = get_experiment_data(path, subjects=start_subject, sessions=session,
                                                       signal_type='raw', input_type=input_type, 
                                                       channels=n_channels, low_cut=20, high_cut=500,
                                                      order=6, window_size=window_size, overlap=overlap, fs=fs, 
                                                      extend=extend, center=center, 
                                                      extend_size=extend_size, whiten=whiten,
                                                      normalize=False, mu=0, ratio=0.2)

####################################################################################
Loading subject 1
####################################################################################
Centred.
Size of the input data is (17152, 64, 102, 1)
The input label shape is (17152,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************


In [5]:
data.shape, label.shape, X_test.shape, y_test.shape

((17152, 64, 102, 1), (17152,), (4288, 64, 102), (4288,))

In [6]:
#Hyper-parameter

input_shape = data.shape[1:]
patch_size = 8
n_patches = 24 
dims = n_patches + 1 
n_transformer_layers = 2
n_heads = 2
transformer_units = [dims * 2,dims]  
mlp_head_units = [128, 34]   
n_classes = len(np.unique(label))
stochastic_depth = False

stochastic_depth_rate = 0.1

n_batches = 32
n_epochs = 150
n_folds = 3
type_of_experiment = 2

In [7]:
model = vit_gesture_classification(input_shape, dims, patch_size, n_patches,
                                  n_transformer_layers, n_heads, transformer_units,
                                   mlp_head_units, n_classes,stochastic_depth,stochastic_depth_rate)

model.summary()

2023-09-03 13:39:09.666882: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-03 13:39:09.699544: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-03 13:39:09.699736: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-03 13:39:09.700222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sEMG-Decomposition"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 102, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 covolution_layer (Sequential)  (None, 32, 51, 25)   20413       ['input_1[0][0]']                
                                                                                                  
 generate__patches (Generate_Pa  (None, None, 1600)  0           ['covolution_layer[0][0]']       
 tches)                                                                                           
                                                                                 

In [ ]:
run_experiment(n_folds, n_batches, n_epochs, start_subject, total_subject, session, path, 
                       input_type,n_channels, window_size,extend, extend_size, center, whiten, ratio, noise_db, std,
                      input_shape, dims, patch_size, n_patches,
                       n_transformer_layers, n_heads, transformer_units,
                       mlp_head_units, n_classes,stochastic_depth,stochastic_depth_rate, type_of_experiment)

************************************************
Experiment Two
************************************************
####################################################################################
Loading subject 1
####################################################################################
Centred.
Size of the input data is (17152, 64, 102, 1)
The input label shape is (17152,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************
Adding noise to RAW input test data
Adding white noise to RAW input test data
---------------------------------------------------
Training for fold 1 -------
Epoch 1/150


2023-09-03 13:39:44.862335: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2023-09-03 13:39:46.024190: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


358/358 [==============================] - 13s 16ms/step - loss: 3.2306 - accuracy: 0.0850
Epoch 2/150
358/358 [==============================] - 6s 16ms/step - loss: 2.3169 - accuracy: 0.2765
Epoch 3/150
358/358 [==============================] - 6s 16ms/step - loss: 1.7529 - accuracy: 0.4245
Epoch 4/150
358/358 [==============================] - 6s 16ms/step - loss: 1.4270 - accuracy: 0.5283
Epoch 5/150
358/358 [==============================] - 6s 16ms/step - loss: 1.2204 - accuracy: 0.5963
Epoch 6/150
358/358 [==============================] - 6s 16ms/step - loss: 1.0951 - accuracy: 0.6321
Epoch 7/150
358/358 [==============================] - 6s 16ms/step - loss: 1.0018 - accuracy: 0.6705
Epoch 8/150
358/358 [==============================] - 6s 16ms/step - loss: 0.9274 - accuracy: 0.6953
Epoch 9/150
358/358 [==============================] - 6s 16ms/step - loss: 0.8556 - accuracy: 0.7200
Epoch 10/150
358/358 [==============================] - 6s 15ms/step - loss: 0.7882 - accurac

358/358 [==============================] - 9s 25ms/step - loss: 1.0004 - accuracy: 0.6740
Epoch 18/150
358/358 [==============================] - 9s 24ms/step - loss: 0.9749 - accuracy: 0.6807
Epoch 19/150
358/358 [==============================] - 9s 24ms/step - loss: 0.9611 - accuracy: 0.6905
Epoch 20/150
358/358 [==============================] - 9s 25ms/step - loss: 0.9293 - accuracy: 0.6949
Epoch 21/150
358/358 [==============================] - 9s 24ms/step - loss: 0.8954 - accuracy: 0.7121
Epoch 22/150
358/358 [==============================] - 9s 24ms/step - loss: 0.8515 - accuracy: 0.7260
Epoch 23/150
358/358 [==============================] - 9s 24ms/step - loss: 0.8469 - accuracy: 0.7293
Epoch 24/150
358/358 [==============================] - 9s 25ms/step - loss: 0.8083 - accuracy: 0.7431
Epoch 25/150
358/358 [==============================] - 9s 25ms/step - loss: 0.7887 - accuracy: 0.7436
Epoch 26/150
358/358 [==============================] - 9s 25ms/step - loss: 0.7657 - 

358/358 [==============================] - 9s 24ms/step - loss: 1.2392 - accuracy: 0.5904
Epoch 13/150
358/358 [==============================] - 9s 24ms/step - loss: 1.1798 - accuracy: 0.6178
Epoch 14/150
358/358 [==============================] - 9s 24ms/step - loss: 1.1425 - accuracy: 0.6240
Epoch 15/150
358/358 [==============================] - 9s 24ms/step - loss: 1.0856 - accuracy: 0.6423
Epoch 16/150
358/358 [==============================] - 9s 24ms/step - loss: 1.0387 - accuracy: 0.6610
Epoch 17/150
358/358 [==============================] - 9s 25ms/step - loss: 0.9969 - accuracy: 0.6760
Epoch 18/150
358/358 [==============================] - 9s 25ms/step - loss: 0.9733 - accuracy: 0.6797
Epoch 19/150
358/358 [==============================] - 9s 24ms/step - loss: 0.9436 - accuracy: 0.6942
Epoch 20/150
358/358 [==============================] - 9s 24ms/step - loss: 0.8945 - accuracy: 0.7127
Epoch 21/150
358/358 [==============================] - 9s 24ms/step - loss: 0.8796 - 

...

355/355 [==============================] - 9s 26ms/step - loss: 0.2646 - accuracy: 0.9190
Epoch 72/150
355/355 [==============================] - 9s 26ms/step - loss: 0.2580 - accuracy: 0.9197
Epoch 73/150
355/355 [==============================] - 9s 25ms/step - loss: 0.2561 - accuracy: 0.9190
Score for fold  2: loss of 0.20588327944278717; accuracy of 93.90200972557068%
---------------------------------------------------
Training for fold 3 -------
Epoch 1/150
355/355 [==============================] - 12s 25ms/step - loss: 3.3320 - accuracy: 0.0742
Epoch 2/150
355/355 [==============================] - 9s 25ms/step - loss: 2.1189 - accuracy: 0.3143
Epoch 3/150
355/355 [==============================] - 9s 25ms/step - loss: 1.7051 - accuracy: 0.4370
Epoch 4/150
355/355 [==============================] - 9s 24ms/step - loss: 1.5191 - accuracy: 0.4925
Epoch 5/150
355/355 [==============================] - 9s 25ms/step - loss: 1.3517 - accuracy: 0.5492
Epoch 6/150
355/355 [=============

355/355 [==============================] - 9s 25ms/step - loss: 0.2362 - accuracy: 0.9267
Epoch 77/150
355/355 [==============================] - 9s 25ms/step - loss: 0.2510 - accuracy: 0.9237
Epoch 78/150
355/355 [==============================] - 9s 25ms/step - loss: 0.2405 - accuracy: 0.9265
Epoch 79/150
355/355 [==============================] - 9s 25ms/step - loss: 0.2433 - accuracy: 0.9236
Epoch 80/150
355/355 [==============================] - 9s 25ms/step - loss: 0.2460 - accuracy: 0.9226
Epoch 81/150
355/355 [==============================] - 9s 25ms/step - loss: 0.2289 - accuracy: 0.9312
Epoch 82/150
355/355 [==============================] - 9s 25ms/step - loss: 0.2367 - accuracy: 0.9277
Epoch 83/150
355/355 [==============================] - 9s 25ms/step - loss: 0.2466 - accuracy: 0.9254
Epoch 84/150
355/355 [==============================] - 9s 25ms/step - loss: 0.2418 - accuracy: 0.9277
Epoch 85/150
355/355 [==============================] - 9s 25ms/step - loss: 0.2514 - 

363/363 [==============================] - 9s 25ms/step - loss: 0.3495 - accuracy: 0.8925
Epoch 45/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3484 - accuracy: 0.8959
Epoch 46/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3468 - accuracy: 0.8956
Epoch 47/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3460 - accuracy: 0.8952
Epoch 48/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3331 - accuracy: 0.9012
Epoch 49/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3299 - accuracy: 0.9014
Epoch 50/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3168 - accuracy: 0.9044
Epoch 51/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3086 - accuracy: 0.9086
Epoch 52/150
363/363 [==============================] - 9s 25ms/step - loss: 0.2998 - accuracy: 0.9088
Epoch 53/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3029 - 

363/363 [==============================] - 9s 25ms/step - loss: 0.2988 - accuracy: 0.9110
Epoch 54/150
363/363 [==============================] - 9s 25ms/step - loss: 0.2967 - accuracy: 0.9111
Epoch 55/150
363/363 [==============================] - 9s 25ms/step - loss: 0.2880 - accuracy: 0.9149
Epoch 56/150
363/363 [==============================] - 9s 25ms/step - loss: 0.2839 - accuracy: 0.9167
Epoch 57/150
363/363 [==============================] - 9s 25ms/step - loss: 0.2929 - accuracy: 0.9113
Epoch 58/150
363/363 [==============================] - 9s 25ms/step - loss: 0.2835 - accuracy: 0.9150
Epoch 59/150
363/363 [==============================] - 9s 25ms/step - loss: 0.2714 - accuracy: 0.9203
Epoch 60/150
363/363 [==============================] - 9s 25ms/step - loss: 0.2709 - accuracy: 0.9183
Epoch 61/150
363/363 [==============================] - 9s 25ms/step - loss: 0.2742 - accuracy: 0.9185
Epoch 62/150
363/363 [==============================] - 9s 25ms/step - loss: 0.2557 - 

363/363 [==============================] - 9s 25ms/step - loss: 0.3366 - accuracy: 0.9010
Epoch 45/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3439 - accuracy: 0.8983
Epoch 46/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3331 - accuracy: 0.9011
Epoch 47/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3165 - accuracy: 0.9084
Epoch 48/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3226 - accuracy: 0.9040
Epoch 49/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3255 - accuracy: 0.9044
Epoch 50/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3144 - accuracy: 0.9070
Epoch 51/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3051 - accuracy: 0.9115
Epoch 52/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3045 - accuracy: 0.9100
Epoch 53/150
363/363 [==============================] - 9s 25ms/step - loss: 0.3052 - 